In [45]:
import os
import pandas as pd
import re
from rapidfuzz import fuzz, process


In [159]:
countries = ['Angola', 'Burundi', 'Cameroon', 'Central African Republic', 'Chad', 'Congo', 'Republic of the Congo', 'DRC', 'Drc', 'DR Congo', 'Equatorial Guinea', 'Gabon']
rename_countries = {
    'Republic Of The Congo': 'Congo',
    'Democratic Republic Of The Congo': 'Drc',
    'Dr Congo': 'Drc'
}

## 1. Load the African Hydropower Atlas

In [116]:
file_atlas = os.path.join(os.getcwd(), 'input', 'African_Hydropower_Atlas_v2-0.xlsx')

In [131]:
def remove_parentheses(text):
    if pd.isna(text):
        return ""
    # Remove all (...) and trim whitespace
    cleaned = re.sub(r"\s*\([^)]*\)", "", text)
    cleaned = re.sub(r"\s+", " ", cleaned).strip()
    return cleaned.lower()

In [137]:
data_atlas = pd.read_excel(file_atlas, sheet_name='1 - Spatial and technical data', index_col=None, skiprows=1, header=1)
data_atlas['Country'] = data_atlas['Country'].str.strip().str.title()
data_atlas['Country'] = data_atlas['Country'].replace(rename_countries)

data_atlas = data_atlas[data_atlas['Country'].isin(countries)]
print(f'Number of countries in the atlas: {len(data_atlas["Country"].unique())}. Countries: {data_atlas["Country"].unique()}')

data_atlas['Unit Name'] = data_atlas['Unit Name'].apply(remove_parentheses)

# data_atlas.set_index(['Country', 'Unit Name'], inplace=True)
print(f'Number of hydropower plants in the atlas: {len(data_atlas)}')



data_flow = pd.read_excel(file_atlas, sheet_name='4a - HydrofleetAll', index_col=None, skiprows=None, header=0)
data_flow = data_flow[data_flow['Country'].isin([c.upper() for c in countries])]
data_flow.rename(columns={'Name': 'Unit Name'}, inplace=True)
data_flow.set_index(['Country', 'Unit Name'], inplace=True)
# Rename columns to match the expected format
cols = pd.MultiIndex.from_product([['baseline', 'dry', 'wet'], range(1, 13)], names=['Scenario', 'Month'])
# Add scenarios to the columns
data_flow = data_flow.set_axis(cols, axis=1)
#print(data_flow.head())

Number of countries in the atlas: 8. Countries: ['Angola' 'Burundi' 'Cameroon' 'Central African Republic' 'Congo' 'Drc'
 'Equatorial Guinea' 'Gabon']
Number of hydropower plants in the atlas: 133


## 2. Load the Global Hydropower Tracker

In [103]:
file_db = os.path.join(os.getcwd(), 'generation', 'input', 'Global-Integrated-Power-April-2025.xlsx')

In [125]:
def clean_plant_name(name: str) -> str:
    if pd.isna(name):
        return ""

    name = name.lower()

    # Phrases to remove
    suffixes = [
        "hydroelectric plant",
        "hydroelectric dam",
        "wind farm",
        "solar farm",
        "solar project",
        "solar plant",
        "solar farms",
        "power station",
        "power plant",
        "hybrid solar farm",
        "thermal power station",
        "thermal plant",
        "central",  # often means "station" in French/Portuguese
        "centrale",
        "centrales d’energie renouvelable",
        "energy project",
        "solar",   # sometimes standalone e.g. "Lubango Solar"
        "dam"
    ]

    # Remove known phrases
    for s in suffixes:
        name = name.replace(s, "")

    # Remove anything in parentheses
    name = re.sub(r"\([^)]*\)", "", name)

    # Remove extra spaces and punctuation
    name = re.sub(r"[^\w\s]", "", name)  # Remove punctuation
    name = re.sub(r"\s+", " ", name)  # Normalize whitespace

    return name.strip()


In [148]:
if os.path.exists(file_db):
    data_raw_tracker = pd.read_excel(file_db, sheet_name='Power facilities', header=[0], index_col=None)
else:
    raise FileNotFoundError(f"File {file_db} does not exist.")

/opt/anaconda3/envs/HydroData/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [162]:


data_tracker = data_raw_tracker[data_raw_tracker['Country/area'].isin(countries)]
data_tracker = data_tracker.rename(columns={'Country/area': 'Country'})
data_tracker['Country'] = data_tracker['Country'].str.strip().str.title()
data_tracker['Country'] = data_tracker['Country'].replace(rename_countries)

print(f'Number of countries in the atlas: {len(data_tracker["Country"].unique())}. Countries: {data_tracker["Country"].unique()}')

data_tracker['Unit Name'] = data_tracker['Plant / Project name']
# Clean the plant names
data_tracker['Unit Name'] = data_tracker['Unit Name'].apply(clean_plant_name)


data_hydro_tracker = data_tracker[
    data_tracker['Type'] == 'hydropower'
].copy()
print(f'Number of hydropower plants in the tracker: {len(data_hydro_tracker)}')

data_non_hydro_tracker = data_tracker[
    ~data_tracker['Type'].str.lower().str.contains('hydropower', na=False)
].copy()
print(f'Number of non-hydropower plants in the tracker: {len(data_non_hydro_tracker)}')

Number of countries in the atlas: 9. Countries: ['Angola' 'Drc' 'Cameroon' 'Central African Republic' 'Chad'
 'Equatorial Guinea' 'Gabon' 'Congo' 'Burundi']
Number of hydropower plants in the tracker: 58
Number of non-hydropower plants in the tracker: 165


## 3. Merge the datasets

In [166]:
# Ensure consistent format
data_atlas.loc[:, 'Name_clean'] = data_atlas.loc[:, 'Unit Name'].str.lower().str.strip()
data_hydro_tracker.loc[:, 'Name_clean'] = data_hydro_tracker.loc[:, 'Unit Name'].str.lower().str.strip()

unified = []
diffs = []

# Loop by country
for country in data_atlas['Country'].unique():
    print(f'Processing country: {country}')
    df_atlas_c = data_atlas[data_atlas['Country'] == country].copy()
    df_tracker_c = data_hydro_tracker[data_hydro_tracker['Country'] == country].copy()

    if df_tracker_c.empty:
        print(f'No tracker data for {country}, skipping...')
        continue

    for _, atlas_row in df_atlas_c.iterrows():
        # Find best match in tracker for this plant

        result = process.extractOne(
            atlas_row['Name_clean'],
            df_tracker_c['Name_clean'],
            scorer=fuzz.token_sort_ratio,
            score_cutoff=60
        )
        if result:
            match_name, score, tracker_idx = result
        else:
            match_name, score, tracker_idx = None, None, None

        if match_name:
            tracker_row = df_tracker_c.loc[tracker_idx]

            # Combine atlas and tracker row into one unified row
            merged_row = {}

            merged_row['Both datasets'] = True

            # Add all columns from atlas row
            for col in data_atlas.columns:
                merged_row[f"{col}_atlas"] = atlas_row[col]

            # Add all columns from tracker row
            for col in data_tracker.columns:
                merged_row[f"{col}_tracker"] = tracker_row[col]

            unified.append(merged_row)
        else:
            merged_row = {f"{col}_atlas": atlas_row[col] for col in data_atlas.columns}
            unified.append(merged_row)

# Step 1: Collect all matched tracker names
matched_tracker_names = set(row[f"Name_clean_tracker"] for row in unified if "Name_clean_tracker" in row)

# Step 2: Filter tracker rows that weren't matched
unmatched_tracker = data_hydro_tracker[
    ~data_hydro_tracker['Name_clean'].isin(matched_tracker_names)
]

# Step 3: Add each unmatched tracker row to unified list (with '_tracker' prefix)
for _, tracker_row in unmatched_tracker.iterrows():
    merged_row = {f"{col}_tracker": tracker_row[col] for col in data_hydro_tracker.columns}
    unified.append(merged_row)


pd.DataFrame(unified).to_csv(os.path.join(os.getcwd(), 'hydro', 'output', 'hydro_atlas_tracker_unified.csv'), index=False)

Processing country: Angola
Processing country: Burundi
No tracker data for Burundi, skipping...
Processing country: Cameroon
Processing country: Central African Republic
Processing country: Congo
Processing country: Drc
Processing country: Equatorial Guinea
Processing country: Gabon
